In [21]:
from __future__ import print_function

import os, sys
module_path = os.path.abspath(os.path.join('../../..'))
sys.path.append(module_path)

import numpy as np
import math
import copy
import pandas as pd
from keras.utils import np_utils
from keras.datasets import mnist
import time
import pickle

from pycrcnn.he.he import TFHEnuFHE
from pycrcnn.he.tfhe_value import TFHEValue
from pycrcnn.he.alu import *

## Dataset

In [22]:
# Min-Max quantization function
def quantize_tensor(x, num_bits, min_val=None, max_val=None):
    if not min_val and not max_val: 
        min_val, max_val = x.min(), x.max()
    qmin = -2.**(num_bits-1)
    qmax = 2.**(num_bits-1) - 1.
   
    x = x - min_val          # Allineo tutto l'array in modo che parta da 0
    x /= (max_val - min_val) # Lo scalo tra 0 e 1    
    x *= (qmax - qmin)       # Lo scalo tra 0 e 16
    x -= qmax                # Lo sfaso tra -8 e 7
    q_x = x.astype(float).round().astype(int)
    
    return q_x

In [23]:
# Prepare Penguins dataset
penguins = pd.read_csv('../penguins_size.csv')
penguins = penguins.sample(frac=1, random_state=2)
penguins = penguins.dropna()

# Feature selection
x_train, y_train = penguins.loc[:, ["island", "culmen_length_mm", "flipper_length_mm", "body_mass_g"]].values, penguins.iloc[:, :1].values

# Encode labels
for i in range(len(y_train)):
    if y_train[i][0] == "Adelie":
        y_train[i][0] = 0
    elif y_train[i][0] == "Gentoo":
        y_train[i][0] = 1
    else:
        y_train[i][0] = 2

island = {}
countI = 0
for i in range(len(x_train)):
  # Island
  if x_train[i][0] in island:
      x_train[i][0] = island[x_train[i][0]]
  else:
      island[x_train[i][0]] = countI
      x_train[i][0] = countI
      countI += 1

# Quantize tensors with 4 bits
x_train[:, 1] = quantize_tensor(x_train[:, 1], 4)
x_train[:, 2] = quantize_tensor(x_train[:, 2], 4)
x_train[:, 3] = quantize_tensor(x_train[:, 3], 4)

# Split Train-Validation set
train, val = 150, 64
x_val, y_val = x_train[train:train+val], y_train[train:train+val]
x_test, y_test = x_train[train+val:], y_train[train+val:]
y_train = np_utils.to_categorical(y_train).astype(int)*8
x_train, y_train = x_train[:train], y_train[:train]

## Net Architecture

In [24]:
SHRT_MAX = 32767
SHRT_MIN = (-SHRT_MAX - 1 )

def isqrt(n):
    x = n
    y = (x + 1) // 2
    while y < x:
        x = y
        y = (x + n // x) // 2
    return x

In [25]:
# DFA WEIGHTS
def DFA_weights_uniform(in_dim, out_dim):
    range = isqrt((12 * SHRT_MAX) / (in_dim + out_dim))
    return np.random.randint(-range, range, (in_dim, out_dim))

In [26]:
# PLA tanh Activation function
def PLA_tanh(act_in, in_dim, out_dim):
    y_max, y_min = 128, -127
    intervals = [128, 75, 32, -31, -74, -127]
    slopes_inv = [y_max, 8, 2, 1, 2, 8, y_max]

    act_out, act_grad_inv  = np.full((act_in.shape[0], out_dim), y_max), np.full((act_in.shape[0], out_dim), slopes_inv[0])

    for i in range(len(act_in)):
        for j in range(len(act_in[i].squeeze())):
            val = act_in[i].squeeze()[j] // ((1 << 8) * in_dim)
            if val < intervals[0]:
                act_out[i][j] = val // 4
                act_grad_inv[i][j] = slopes_inv[1]
            if val < intervals[1]:
                act_out[i][j] = val
                act_grad_inv[i][j] = slopes_inv[2]
            if val < intervals[2]:
                act_out[i][j] = val * 2
                act_grad_inv[i][j] = slopes_inv[3]
            if val < intervals[3]:
                act_out[i][j] = val
                act_grad_inv[i][j] = slopes_inv[4]
            if val < intervals[4]:
                act_out[i][j] = val // 4
                act_grad_inv[i][j] = slopes_inv[5]
            if val < intervals[5]:
                act_out[i][j] = y_min
                act_grad_inv[i][j] = slopes_inv[6]
    return act_out.astype(int), act_grad_inv

In [27]:
# L2 Loss Function
def L2(y_true, net_out):
    loss = np.zeros((y_true.shape[0], y_true.shape[1]))
    for i in range(len(y_true)):
        for j in range(len(y_true[i])):
            loss[i][j] = net_out[i].squeeze()[j] - y_true[i][j]
    return loss.astype(int)

In [28]:
# Manual matmul used to check overflow
def matmul(m1, m2, check_overflow=False, bits_tfhe = 0, clip_overflow=False, layer=""):
    max_value = 2.**(bits_tfhe-1) - 1.
    min_value = -2.**(bits_tfhe-1)
    res = [[0 for i in range(len(m2[0]))] for j in range(len(m1))]

    for i in range(len(m1)):
        for j in range(len(m2[0])):
            for k in range(len(m2)):
                mul = m1[i][k] * m2[k][j]

                if clip_overflow:
                    mul = max_value if mul>max_value else mul
                    mul = min_value if mul<min_value else mul

                if check_overflow:
                    overflow = 1 if mul>max_value else 0
                    overflow += 1 if mul<min_value else 0
                    if overflow > 0:
                        print("MUL: matmul overflow layer: " + layer)

                res[i][j] += mul

                if clip_overflow:
                    res[i][j] = max_value if res[i][j]>max_value else res[i][j]
                    res[i][j] = min_value if res[i][j]<min_value else res[i][j]

                if check_overflow:
                    overflow = 1 if res[i][j]>max_value else 0
                    overflow += 1 if res[i][j]<min_value else 0
                    if overflow > 0:
                        print("ADD: matmul overflow layer: " + layer)
    return np.array(res)

In [29]:
# MaxPool Layer
class MaxPoolLayer:
    def __init__(self, kernel_size, stride=(1, 1)):
        self.kernel_size = kernel_size
        self.stride = stride

    def forward(self, batch, check_overflow=False, bits_tfhe = 0, clip_overflow=False):
        return np.array([_max(image, self.kernel_size, self.stride) for image in batch])

    def backward(self, loss, lr_inv, check_overflow=False, bits_tfhe = 0, clip_overflow=False):
        return loss

def _max(image, kernel_size, stride):
    x_s = stride[1]
    y_s = stride[0]

    x_k = kernel_size[1]
    y_k = kernel_size[0]

    # print(image)
    x_d = len(image[0])
    y_d = len(image)

    x_o = ((x_d - x_k) // x_s) + 1
    y_o = ((y_d - y_k) // y_s) + 1

    def get_submatrix(matrix, x, y):
        index_row = y * y_s
        index_column = x * x_s
        return matrix[index_row: index_row + y_k, index_column: index_column + x_k]

    return [[np.max(get_submatrix(image, x, y).flatten()) for x in range(0, x_o)] for y in range(0, y_o)]

In [30]:
# Flatten Layer
class FlattenLayer:
    def __init__(self):
        pass

    def forward(self, flatten_in, check_overflow=False, bits_tfhe = 0, clip_overflow=False):
        return flatten_in.reshape(flatten_in.shape[0], flatten_in.shape[1]*flatten_in.shape[2])

    def backward(self, loss, lr_inv, check_overflow=False, bits_tfhe = 0, clip_overflow=False):
        return loss

In [31]:
# FC Layer
class FCLayer:
    def __init__(self, in_dim, out_dim, last_layer = False):
        self.in_dim, self.out_dim = in_dim, out_dim
        self.last_layer = last_layer
        self.weights = np.zeros((in_dim, out_dim)).astype(int)
        self.bias = np.zeros((1, out_dim)).astype(int)
        self.DFA_weights = np.zeros((1, 1)).astype(int)
    
    def forward(self, fc_in, check_overflow=False, bits_tfhe = 0, clip_overflow=False):
        max_value = 2.**(bits_tfhe-1) - 1.
        min_value = -2.**(bits_tfhe-1)
        layer = "fw input="  + repr(self.in_dim)
        self.input = fc_in

        if check_overflow or clip_overflow:
            dot = matmul(self.input, self.weights, check_overflow, bits_tfhe, clip_overflow, layer) + self.bias
        else:
            dot = (self.input @ self.weights) + self.bias

        if clip_overflow:
            dot = np.clip(dot, min_value, max_value)
        
        if check_overflow:
            overflow = (dot[dot>max_value]).size
            overflow += (dot[dot<min_value]).size
            if overflow > 0:
                print("ADD: Bias overflow layer: "  + layer)
                print(overflow)
        
        output, self.act_grad_inv = PLA_tanh(dot, self.in_dim, self.out_dim)
        return output
    
    def backward(self, loss, lr_inv, check_overflow=False, bits_tfhe=0, clip_overflow=False):   
        max_value = 2.**(bits_tfhe-1) - 1.
        min_value = -2.**(bits_tfhe-1)
        layer = "bw input="  + repr(self.in_dim)

        d_DFA = self.compute_dDFA(loss, lr_inv, check_overflow, bits_tfhe, clip_overflow, layer)

        if check_overflow:
            overflow = (d_DFA[d_DFA>max_value]).size
            overflow += (d_DFA[d_DFA<min_value]).size
            if overflow > 0:
                print("Deltas overflow layer: " + layer)
                print(overflow)
        
        if check_overflow or clip_overflow:
            weights_update = matmul(self.input.T, d_DFA, check_overflow, bits_tfhe, clip_overflow, layer)
        else:
            weights_update = self.input.T @ d_DFA
        
        if check_overflow:
            overflow = (weights_update[weights_update>max_value]).size
            overflow += (weights_update[weights_update<min_value]).size
            if overflow > 0:
                print("Weights Update overflow layer: " + layer)
                print(overflow)
        
        weights_update = (weights_update // lr_inv).astype(int)
        self.weights -= weights_update

        if check_overflow:
            overflow = (self.weights[self.weights>max_value]).size
            overflow += (self.weights[self.weights<min_value]).size
            if overflow > 0:
                print("ADD: weights overflow layer: " + layer)
                print(overflow)
        
        ones = np.ones((len(d_DFA), 1)).astype(int)
        if check_overflow or clip_overflow:
            bias_update = matmul(d_DFA.T, ones, check_overflow, bits_tfhe, clip_overflow, layer)
        else:
            bias_update = d_DFA.T @ ones
        
        if check_overflow:
            overflow = (bias_update[bias_update>max_value]).size
            overflow += (bias_update[bias_update<min_value]).size
            if overflow > 0:
                print("Bias Update overflow layer: " + layer)
                print(overflow)
        
        bias_update = (bias_update.T // lr_inv).astype(int)
        self.bias -= bias_update

        if check_overflow:
            overflow = (self.bias[self.bias>max_value]).size
            overflow += (self.bias[self.bias<min_value]).size
            if overflow > 0:
                print("ADD: bias overflow layer: " + layer)
                print(overflow)
        
        return loss
    
    def compute_dDFA(self, loss, lr_inv, check_overflow=False, bits_tfhe=0, clip_overflow=False, layer=""):
        if self.last_layer:
            d_DFA = np.floor_divide(loss, self.act_grad_inv)
        else:
            if self.DFA_weights.shape[0] != loss.shape[1] and  self.DFA_weights.shape[1] != self.weights.shape[1]: # 0 rows, 1 cols
                print("DFA not initialized!")
            if check_overflow or clip_overflow:
                dot = matmul(loss, self.DFA_weights, check_overflow, bits_tfhe, clip_overflow, layer)
            else:
                dot = loss @ self.DFA_weights
            d_DFA = np.floor_divide(dot, self.act_grad_inv)
        return d_DFA

In [32]:
# Network
class Network:
    def __init__(self):
        self.layers = []

    # add layer to network
    def add(self, layer):
        self.layers.append(layer)

    # Test
    def test(self, x_test, y_test, check_overflow=False, bits_tfhe=0, clip_overflow=False):
        corr = 0
        for j in range(len(x_test)):
            pred = self.predict(x_test[j], check_overflow, bits_tfhe, clip_overflow)
            if pred == y_test[j]:
                corr += 1
        return corr / len(x_test) * 100

    # Predict output
    def predict(self, input_data, check_overflow=False, bits_tfhe=0, clip_overflow=False):
        output = np.expand_dims(input_data, axis=0)
        for layer in self.layers:
            output = layer.forward(output, check_overflow, bits_tfhe, clip_overflow)
        return output.argmax()

    # train the network
    def fit(self, x_train, y_train, epochs, mini_batch_size, lr_inv, check_overflow=False, bits_tfhe=0, clip_overflow=False):
        train_accs, val_accs = [], []
        for i in range(epochs):
            epoch_corr = 0
            for j in range(int(len(x_train)/mini_batch_size)):
                batch_corr = 0
                idx_start = j * mini_batch_size
                idx_end = idx_start + mini_batch_size

                batch_in = x_train[idx_start:idx_end]
                batch_target = y_train[idx_start:idx_end]

                # Forward propagation
                for layer in self.layers:
                  batch_in = layer.forward(batch_in, check_overflow, bits_tfhe, clip_overflow)
                fwd_out = batch_in               

                # Loss
                loss = L2(batch_target, fwd_out)

                for r in range(mini_batch_size):
                    if batch_target[r].argmax() == fwd_out[r].argmax():
                        batch_corr += 1
                
                # Backward propagation
                for layer in reversed(self.layers):
                    layer.backward(loss, lr_inv, check_overflow, bits_tfhe, clip_overflow)
                
                epoch_corr += batch_corr

            acc = epoch_corr/len(x_train) * 100
            train_accs.append(acc)
            val_accs.append(self.test(x_val, y_val))
            
        return train_accs, val_accs

## Results

In [33]:
## UPLOADE DFA WEIGHTS
DFA_weights = np.load("res/DFA_weights1.npy")

### Model 1

In [34]:
# Load decrypted trained weights TFHE-NN-1
with open("res/trained_weights1.pkl", "rb") as f:
    decrypted_weights1_1 = pickle.load(f)
    decrypted_bias1_1 = pickle.load(f)
    decrypted_weights2_1 = pickle.load(f)
    decrypted_bias2_1 = pickle.load(f)
    decrypted_corr1 = pickle.load(f)

In [35]:
# Trained TFHE-NN-1 with decryted trained weights on encrypted data
trained_net1 = Network()
trained_net1.add(FCLayer(4, 2))
trained_net1.add(FCLayer(2, 3, last_layer=True))

trained_net1.layers[0].DFA_weights = DFA_weights

trained_net1.layers[0].weights = decrypted_weights1_1
trained_net1.layers[0].bias = decrypted_bias1_1
trained_net1.layers[1].weights = decrypted_weights2_1
trained_net1.layers[1].bias = decrypted_bias2_1

print("Decrypted validation accuracy TFHE-NN-1: " + repr(decrypted_corr1*100/len(x_val)) + " %")

Decrypted validation accuracy TFHE-NN-1: 92.1875 %


### Model 2

In [36]:
# Load decrypted trained weights TFHE-NN-2
with open("res/trained_weights2.pkl", "rb") as f:
    decrypted_weights1_2 = pickle.load(f)
    decrypted_bias1_2 = pickle.load(f)
    decrypted_weights2_2 = pickle.load(f)
    decrypted_bias2_2 = pickle.load(f)
    decrypted_corr2 = pickle.load(f)

In [37]:
# Trained TFHE-NN-2 with decryted trained weights on encrypted data
trained_net2 = Network()
trained_net2.add(FCLayer(4, 2))
trained_net2.add(FCLayer(2, 3, last_layer=True))

trained_net2.layers[0].DFA_weights = DFA_weights

trained_net2.layers[0].weights = decrypted_weights1_2
trained_net2.layers[0].bias = decrypted_bias1_2
trained_net2.layers[1].weights = decrypted_weights2_2
trained_net2.layers[1].bias = decrypted_bias2_2

print("Decrypted validation accuracy TFHE-NN-2: " + repr(decrypted_corr2*100/len(x_val)) + " %")

Decrypted validation accuracy TFHE-NN-2: 87.5 %


### Model 3

In [38]:
# Load decrypted trained weights TFHE-NN-3
with open("res/trained_weights3.pkl", "rb") as f:
    decrypted_weights1_3 = pickle.load(f)
    decrypted_bias1_3 = pickle.load(f)
    decrypted_weights2_3 = pickle.load(f)
    decrypted_bias2_3 = pickle.load(f)
    decrypted_corr3 = pickle.load(f)

In [39]:
# Trained TFHE-NN-3 with decryted trained weights on encrypted data
trained_net3 = Network()
trained_net3.add(FCLayer(4, 2))
trained_net3.add(FCLayer(2, 3, last_layer=True))

trained_net3.layers[0].DFA_weights = DFA_weights

trained_net3.layers[0].weights = decrypted_weights1_3
trained_net3.layers[0].bias = decrypted_bias1_3
trained_net3.layers[1].weights = decrypted_weights2_3
trained_net3.layers[1].bias = decrypted_bias2_3

print("Decrypted validation accuracy TFHE-NN-3: " + repr(decrypted_corr3*100/len(x_val)) + " %")

Decrypted validation accuracy TFHE-NN-3: 85.9375 %


### Model 4

In [40]:
# Load decrypted trained weights TFHE-NN-4
with open("res/trained_weights4.pkl", "rb") as f:
    decrypted_weights1_4 = pickle.load(f)
    decrypted_bias1_4 = pickle.load(f)
    decrypted_weights2_4 = pickle.load(f)
    decrypted_bias2_4 = pickle.load(f)
    decrypted_corr4 = pickle.load(f)

In [41]:
# Trained TFHE-NN-4 with decryted trained weights on encrypted data
trained_net4 = Network()
trained_net4.add(FCLayer(4, 2))
trained_net4.add(FCLayer(2, 3, last_layer=True))

trained_net4.layers[0].DFA_weights = DFA_weights

trained_net4.layers[0].weights = decrypted_weights1_4
trained_net4.layers[0].bias = decrypted_bias1_4
trained_net4.layers[1].weights = decrypted_weights2_4
trained_net4.layers[1].bias = decrypted_bias2_4

print("Decrypted validation accuracy TFHE-NN-4: " + repr(decrypted_corr4*100/len(x_val)) + " %")

Decrypted validation accuracy TFHE-NN-4: 84.375 %


### CV Resulting Model

In [42]:
# Load decrypted cross validated weights
with open("res/cross_validated_weights.pkl", "rb") as f:
    decrypted_weights1_cv = pickle.load(f)
    decrypted_bias1_cv = pickle.load(f)
    decrypted_weights2_cv = pickle.load(f)
    decrypted_bias2_cv = pickle.load(f)

In [43]:
# Resulting CV TFHE-NN with decryted cross validated weights
cv_net = Network()
cv_net.add(FCLayer(4, 2))
cv_net.add(FCLayer(2, 3, last_layer=True))

cv_net.layers[0].DFA_weights = DFA_weights

cv_net.layers[0].weights = decrypted_weights1_cv
cv_net.layers[0].bias = decrypted_bias1_cv
cv_net.layers[1].weights = decrypted_weights2_cv
cv_net.layers[1].bias = decrypted_bias2_cv

In [44]:
val_acc_cv = cv_net.test(x_val, y_val, check_overflow=True, bits_tfhe=16)
print("Validation accuracy CV TFHE-NN: " + repr(val_acc_cv) + " %")

Validation accuracy CV TFHE-NN: 92.1875 %


In [49]:
test_acc_cv = cv_net.test(x_test, y_test, check_overflow=True, bits_tfhe=16)
print("Test accuracy CV TFHE-NN: " + repr(test_acc_cv) + " %")

Test accuracy CV TFHE-NN: 93.33333333333333 %


### L2 Norms Comparison

In [46]:
# Compute L2 norm of learned weights of the 4 models on encrypted data
norm_encrypted_weights, norm_plain_weights = 0, 0

L2_norms = []
for net in [trained_net1, trained_net2, trained_net3, trained_net4]:
  norm = 0
  for l in net.layers:
      if hasattr(l, "weights"):
          norm += np.linalg.norm(l.weights)
          norm += np.linalg.norm(l.bias)
  L2_norms.append(norm)

In [47]:
# Compute L2 norm of cross validated weights on encrypted data
norm_cv_net = 0
for l in cv_net.layers:
    norm_cv_net += np.linalg.norm(l.weights)
    norm_cv_net += np.linalg.norm(l.bias)

In [53]:
for i in range(len(L2_norms)):
    print("Difference L2 norms TFHE-NN-" + str(i+1) + " vs. CV TFHE-NN: " + str(L2_norms[i] - norm_cv_net))

Difference L2 norms TFHE-NN-1 vs. CV TFHE-NN: 0.0
Difference L2 norms TFHE-NN-2 vs. CV TFHE-NN: -414.5183905686772
Difference L2 norms TFHE-NN-3 vs. CV TFHE-NN: -1112.5233572468514
Difference L2 norms TFHE-NN-4 vs. CV TFHE-NN: -2595.853748717702
